In [21]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import *
from keras.models import Sequential, Model
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
import librosa
sr = 16000
batch_size = 32
max_size = 79872
import matplotlib.pyplot as plt
import soundfile as sf

In [25]:
files = os.listdir('./noises')
for file in files:
    noise_class = file.split("_")[0]
    if os.path.exists('./splits/'+noise_class) is False:
        os.makedirs('./splits/'+noise_class)
    
    extension = "."+file.split(".")[-1]
    segment =  "./splits/"+noise_class+'/'+file.split(extension)[0]
    command = 'ffmpeg -i ./noises/'+file+' -f segment -segment_time 5 -c copy "'+segment+'%03d"'+extension
#     print(command)
    print(command)
    os.system(command)

ffmpeg -i ./noises/Construction_arafat_1h30m.m4a -f segment -segment_time 5 -c copy "./splits/Construction/Construction_arafat_1h30m%03d".m4a
ffmpeg -i ./noises/Construction_arafat_25m.m4a -f segment -segment_time 5 -c copy "./splits/Construction/Construction_arafat_25m%03d".m4a
ffmpeg -i ./noises/Construction_arafat_2h2min.m4a -f segment -segment_time 5 -c copy "./splits/Construction/Construction_arafat_2h2min%03d".m4a
ffmpeg -i ./noises/Construction_arafat_48m.m4a -f segment -segment_time 5 -c copy "./splits/Construction/Construction_arafat_48m%03d".m4a
ffmpeg -i ./noises/Crowd_1_arafat_3h2min.m4a -f segment -segment_time 5 -c copy "./splits/Crowd/Crowd_1_arafat_3h2min%03d".m4a
ffmpeg -i ./noises/Crowd_2_arafat_2h15m.m4a -f segment -segment_time 5 -c copy "./splits/Crowd/Crowd_2_arafat_2h15m%03d".m4a
ffmpeg -i ./noises/Traffic_15062023_Rhythm.mp3 -f segment -segment_time 5 -c copy "./splits/Traffic/Traffic_15062023_Rhythm%03d".mp3
ffmpeg -i ./noises/Traffic_Noise.mp3 -f segment -segm

In [27]:
n_fft = 800
hop_length = 200
sr = 16000

def stft_loader(loaded):
    stft = librosa.stft(loaded, n_fft=n_fft, hop_length=hop_length)
    stft = np.float32(np.real(stft))
    return stft

def mel_loader(loaded):
    mel = librosa.feature.melspectrogram(y=loaded, n_fft=n_fft, hop_length=hop_length, sr=sr)
    mel = np.float32(np.real(mel))
    return mel

def mfcc_loader(loaded):
    mfcc = librosa.feature.mfcc(y=loaded, n_fft=n_fft, hop_length=hop_length, sr=sr, n_mfcc=128)
    mfcc = np.float32(np.real(mfcc))
    return mfcc

In [29]:
def padarray(A, size):
    t = size - len(A)
    return np.pad(A, pad_width=(0, t), mode='constant')

In [31]:
X = []
y = []
# os.makedirs('./stft')
# os.makedirs('./mel')
os.makedirs('./mfcc')
for i, td in enumerate(os.listdir('./splits')):
    label = i
    files = os.listdir('./splits/'+td)
    for k in range(len(files)):
#         mel_dir = './mel/'+td +'/'
#         stft_dir = './stft/'+td +'/'
        mfcc_dir = './mfcc/'+td +'/'
        
        path = './splits/'+td +'/'+ files[k]
        
        loaded, sr = librosa.load(path, sr=16000)
        if loaded.shape[0] < max_size:
            loaded = padarray(loaded, max_size)
        loaded = loaded[0:max_size]
        
#         stft = stft_loader(loaded)
#         mel = mel_loader(loaded)
        mfcc = mfcc_loader(loaded)
        
#         stft = librosa.util.normalize(stft)
#         mel = librosa.util.normalize(mel)
        mfcc = librosa.util.normalize(mfcc)
        
    
        
#         if os.path.exists(stft_dir) is False:
#             os.makedirs(stft_dir)
            
#         if os.path.exists(mel_dir) is False:
#             os.makedirs(mel_dir)
                
        if os.path.exists(mfcc_dir) is False:
            os.makedirs(mfcc_dir)
            
#         if stft.shape[0] == 401 and stft.shape[1] == 400:
#             np.save(stft_dir+files[k], stft)
            
#         if mel.shape[0] == 128 and mel.shape[1] == 400:
#             np.save(mel_dir+files[k], mel)
            
        if mfcc.shape[0] == 128 and mfcc.shape[1] == 400:
            np.save(mfcc_dir+files[k], mfcc) 
            
        else:
            os.remove(path)
            print("Deleting (Less than 5 seconds audio file): ", path)

C:\Users\muaz\AppData\Local\Temp\ipykernel_13192\1826686184.py:16: UserWarning: PySoundFile failed. Trying audioread instead.
  loaded, sr = librosa.load(path, sr=16000)
